In [2]:
import numpy as np
import math

In [3]:
def mod2(A):
    """Reduce a numpy integer matrix A modulo 2."""
    return np.remainder(A, 2)

def build_2d_matrices(delta0):
    k, j = delta0.shape
    # For simplicity, let "1" in Eq. (43) be the identity of matching dimension.
    # You may need to adjust which dimension is used, depending on how delta0 is shaped.
    I_k = np.eye(k, dtype=int)  # Identity matching # of rows of H
    I_j = np.eye(j, dtype=int)  # Identity matching # of columns of H

    # Eq. (43):
    #  tilde_delta_{-1} = [ 1 ⊗ delta0^T
    #                      delta0 ⊗ 1    ]
    # print(np.kron(I_j, delta0.T).shape)
    # print(np.kron(delta0, I_k).shape)
    delta_m1_2d = np.block([
        [np.kron(I_j, delta0.T)],
        [np.kron(delta0, I_k)]
    ])
    
    #  tilde_delta_0 = [ delta0 ⊗ 1   |   1 ⊗ delta0^T ]
    delta_0_2d = np.block([
        [np.kron(delta0, I_j), np.kron(I_k, delta0.T)]
    ])

    return mod2(delta_m1_2d), mod2(delta_0_2d)

def build_4d_matrices(delta_m1_2d, delta_0_2d):
    """
    Parameters
    ----------
    delta_m1_2d : The matrix δ₋₁ from previous construction.
    delta_0_2d : The matrix δ₀ from previous construction.

    Returns
    -------
    mz : Eqn. (58)
    hz : Eqn. (59)
    hx : Eqn. (60)
    mx : Eqn. (61) 
    """
    k, j = delta_0_2d.shape
    print(f'k, j: {k}, {j}')

    # (58) δ₋₂ =  ( I ⊗ δ₀ᵀ )
    #             ( δ₋₁ ⊗ I  )
    mz = np.vstack([
        np.kron(np.eye(k), delta_0_2d.T),
        np.kron(delta_m1_2d, np.eye(k))
    ])

    # (59) δ₋₁(new):
    top_left  = np.kron(np.eye(k), delta_m1_2d.T)
    top_right = np.zeros_like(top_left)
    mid_left  = np.kron(delta_m1_2d, np.eye(j))
    mid_right = np.kron(np.eye(j), delta_0_2d.T)
    bot_right = np.kron(delta_0_2d, np.eye(k))
    bot_left = np.zeros_like(bot_right)
    hz = np.block([
        [top_left,  top_right],
        [mid_left,  mid_right],
        [bot_left,  bot_right]
    ])

    # (60) δ₀(new) :
    top_left  = np.kron(delta_m1_2d, np.eye(k))
    top_mid = np.kron(np.eye(j), delta_m1_2d.T)
    top_right = np.zeros_like(top_left)
    bot_left  = np.zeros_like(top_left)
    bot_mid = np.kron(delta_0_2d, np.eye(j))
    bot_right = np.kron(np.eye(k), delta_0_2d.T)
    hx = np.block([
        [top_left, top_mid, top_right],
        [bot_left,  bot_mid, bot_right]
    ])

    # (61) δ₁(new):
    mx =  np.hstack([
        np.kron(delta_0_2d, np.eye(k)),
        np.kron(np.eye(k), delta_m1_2d.T)
    ])

    return mz, hz, hx, mx

In [4]:
def verify_delta_condition(delta_i, delta_i_minus_1):
    """
    Verify that δ_i * δ_{i-1} = 0 mod 2.
    """
    product = np.mod(np.dot(delta_i, delta_i_minus_1), 2)  # Compute δ_i * δ_{i-1} mod 2
    return np.all(product == 0)  # Check if all entries are zero

delta0_works = np.array([[1, 1, 0], 
                         [0, 1, 1]], dtype=float)


delta0_fails = np.array([[1, 1, 0], 
                           [0, 1, 1,], 
                           [1, 0, 1]], dtype=float)

delta0_try = np.array([[1, 1, 0, 0], 
                       [0, 1, 1, 0], 
                       [0, 0, 1, 1]], dtype=float)

for delta0 in [delta0_works, delta0_fails, delta0_try]:
    print(f'input matrix shape: {delta0.shape}')
    tdm1, td0 = build_2d_matrices(delta0)
    print(f'2d matrices: {tdm1.shape, td0.shape}')
    dm2, dm1_new, d0_new, d1_new = build_4d_matrices(tdm1, td0)
    print("delta_{-2}:", dm2.shape)
    print("delta_{-1} (new):", dm1_new.shape,)
    print("delta_{0}  (new):", d0_new.shape)
    print("delta_{1}  (new):", d1_new.shape)

    # Verify condition δ_{i} * δ_{i-1} = 0 mod 2 for each consecutive pair
    print("δ_{-2} * δ_{-1} ≡ 0 mod 2:", verify_delta_condition(dm1_new, dm2))
    print("δ_{-1} * δ_{0} ≡ 0 mod 2:", verify_delta_condition(d0_new, dm1_new))
    print("δ_{0} * δ_{1} ≡ 0 mod 2:", verify_delta_condition(d1_new, d0_new))
    print()

input matrix shape: (2, 3)
2d matrices: ((13, 6), (6, 13))
k, j: 6, 13
delta_{-2}: (156, 36)
delta_{-1} (new): (241, 156)
delta_{0}  (new): (156, 241)
delta_{1}  (new): (36, 156)
δ_{-2} * δ_{-1} ≡ 0 mod 2: True
δ_{-1} * δ_{0} ≡ 0 mod 2: True
δ_{0} * δ_{1} ≡ 0 mod 2: True

input matrix shape: (3, 3)
2d matrices: ((18, 9), (9, 18))
k, j: 9, 18
delta_{-2}: (324, 81)
delta_{-1} (new): (486, 324)
delta_{0}  (new): (324, 486)
delta_{1}  (new): (81, 324)
δ_{-2} * δ_{-1} ≡ 0 mod 2: True
δ_{-1} * δ_{0} ≡ 0 mod 2: True
δ_{0} * δ_{1} ≡ 0 mod 2: True

input matrix shape: (3, 4)
2d matrices: ((25, 12), (12, 25))
k, j: 12, 25
delta_{-2}: (600, 144)
delta_{-1} (new): (913, 600)
delta_{0}  (new): (600, 913)
delta_{1}  (new): (144, 600)
δ_{-2} * δ_{-1} ≡ 0 mod 2: True
δ_{-1} * δ_{0} ≡ 0 mod 2: True
δ_{0} * δ_{1} ≡ 0 mod 2: True

